# RED SOCIAL (CONSULTAS) #


**Utilizando la BD 'Social' **

In [1]:
USE SOCIAL

Commands completed successfully.

Total execution time: 00:00:00.477

# Basic SELECT, Table Variables, Set Operators

**1. Friends of Gabes: : Find the names of all students who are friends with someone named Gabriel.**

In [5]:
SELECT h2.name as [Alumno] FROM Friend 
 INNER JOIN  Highschooler h1 on Friend.ID1 = h1.ID
 INNER JOIN Highschooler h2 on Friend.ID2 = h2.ID
 WHERE H1.name='Gabriel'
 ORDER BY h2.name

(5 rows affected)

Total execution time: 00:00:00.090

Alumno
Alexis
Andrew
Cassandra
Jessica
Jordan


**2.  Like, scandalous!: For every student who likes someone 2 or more grades below their own grade, return that student's name and grade, and the name and grade of the student they like.**

In [6]:
SELECT h1.name as [Student 1 Name], h1.grade as [Student 1 Grade],h2.name as [Student 2 Name], h2.grade as [Student 2 Grade] 
    FROM Highschooler h1, Likes L1, Highschooler h2 
    WHERE h1.ID = L1.ID1 AND h2.ID = L1.ID2 AND (h1.grade-h2.grade)>=2;



(1 row affected)

Total execution time: 00:00:00.211

Student 1 Name,Student 1 Grade,Student 2 Name,Student 2 Grade
John,12,Haley,10


**3.  Should totally be dating: For every pair of students who both like each other, return the name and grade of both students. Include each pair only once.**

In [8]:
SELECT H1.name, H1.grade, H2.name, H2.grade FROM Highschooler H1, Highschooler H2, Likes L1, Likes L2  
WHERE (H1.ID  = L1.ID1) AND
      (H2.ID  = L1.ID2) AND  
      (L1.ID1 = L2.ID2) AND
      (L1.ID2 = L2.ID1) AND 
      (L1.ID1 < L1.ID2)

(2 rows affected)

Total execution time: 00:00:00.146

name,grade,name,grade
Gabriel,9,Cassandra,9
Jessica,11,Kyle,12


**4.  Alias: For every situation where student A likes student B, but student B likes a different student C, return the names and grades of A, B, and C**

In [7]:
SELECT H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade 
    FROM Highschooler H1, Highschooler H2, Highschooler H3, Likes L1, Likes L2 
    WHERE (H1.ID = L1.ID1)  AND 
          (H2.ID = L1.ID2)  AND 
          (H3.ID = L2.ID2)  AND 
          (L1.ID2 = L2.ID1) AND 
          (L1.ID1 <> L2.ID2)


(2 rows affected)

Total execution time: 00:00:01.496

name,grade,name,grade,name,grade
Andrew,10,Cassandra,9,Gabriel,9
Gabriel,11,Alexis,11,Kris,10


**5.  Need more intel: Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade. (Hint: Without subqueries, you will need to use the EXCEPT operator.)**

In [9]:
SELECT name, grade from Highschooler H1 
      EXCEPT SELECT name, grade from Highschooler H2,Likes L 
      WHERE L.ID1=H2.ID OR L.ID2=H2.ID
ORDER BY grade, name


(3 rows affected)

Total execution time: 00:00:00.147

name,grade
Jordan,9
Tiffany,9
Logan,12


# Subqueries in WHERE, FROM, SELECT#


**6.  Have a chance: For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades.**

In [10]:
SELECT H1.name, H1.grade, H2.name, H2.grade FROM Highschooler H1, Highschooler H2, Likes L1 
WHERE   H1.ID = L1.ID1 AND 
        H2.ID = L1.ID2 AND 
        L1.ID2 not in (select ID1 from Likes);


(4 rows affected)

Total execution time: 00:00:00.262

name,grade,name,grade
John,12,Haley,10
Austin,11,Jordan,12
Alexis,11,Kris,10
Brittany,10,Kris,10


**7.  Narrow-minded: Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade.**

In [11]:
SELECT H1.name, H1.grade FROM Highschooler H1 
WHERE H1.ID 
not in (SELECT H2.ID FROM Highschooler H2, Friend F1, Highschooler H3 
        WHERE H2.ID = F1.ID1 AND H3.ID = F1.ID2 AND H2.grade <> H3.grade) 
ORDER BY grade, name;


(7 rows affected)

Total execution time: 00:00:00.218

name,grade
Jordan,9
Brittany,10
Haley,10
Kris,10
Gabriel,11
John,12
Logan,12


**8.  Broad-minded: Find names and grades of students who only have friends in different grades.**

In [13]:
--SE USO 'ALL' PARA ESPECIIFICAR QUE LA SUBCONSULTA DEBE CUMPLIR CON TODAS LAS CONDICIONES--
SELECT DISTINCT H1.name,H1.grade FROM Highschooler H1, Friend F1
WHERE H1.ID = F1.ID1 
      AND H1.grade <> ALL(SELECT H2.grade FROM Highschooler H2, Friend F2 WHERE F2.ID1 = F1.ID1 AND H2.ID = F2.ID2)


(1 row affected)

Total execution time: 00:00:00.116

name,grade
Austin,11


**9.  Could you introduce me? For each student A who likes a student B where the two are not friends, find if they have a friend C in common. For all such trios, return the name and grade of A, B, and C.**

In [14]:
SELECT H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade FROM Highschooler H1, Highschooler H2, Highschooler H3, Likes L1, Friend F1, Friend F2 
WHERE (H1.ID = L1.ID1) AND (H2.ID = L1.ID2) AND (F1.ID1 = H3.ID) AND (F1.ID2 = H1.ID) AND (F2.ID1 = H3.ID) AND (F2.ID2 = H2.ID) 
AND H1.ID not in (SELECT ID1 FROM Friend WHERE ID2 = H2.ID);


(3 rows affected)

Total execution time: 00:00:00.277

name,grade,name,grade,name,grade
Austin,11,Jordan,12,Andrew,10
Austin,11,Jordan,12,Kyle,12
Andrew,10,Cassandra,9,Gabriel,9


# Aggregation #

**10.  Duplication: Find the difference between the number of students in the school and the number of different first names.**

In [15]:
SELECT count(ID) - count(distinct name) FROM Highschooler;

(1 row affected)

Total execution time: 00:00:00.101

(No column name)
2


**11.  Average affability: What is the average number of friends per student?**

In [16]:
SELECT avg(friends) as Average FROM (select ID1,count(ID2) as friends from Friend group by ID1) SBQ;


(1 row affected)

Total execution time: 00:00:00.032

Average
2


**12.  Social circle: Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count Cassandra, even though technically she is a friend of a friend.**

In [17]:
DECLARE @tabla1 as TABLE (ID int)
DECLARE @tabla2 as TABLE (ID2 int)

INSERT INTO @tabla1 SELECT ID2 FROM Friend WHERE ID1 = 
(SELECT ID FROM Highschooler WHERE name = 'Cassandra')

INSERT INTO @tabla2 SELECT DISTINCT ID2 FROM Friend
 WHERE ID1 in (SELECT * FROM @tabla1) AND ID2 
 <> (SELECT ID FROM Highschooler WHERE name = 'Cassandra')

SELECT count(ID) FROM ((SELECT * FROM @tabla1) UNION (SELECT * FROM @tabla2)) SBQ;

(2 rows affected)

(5 rows affected)

(1 row affected)

Total execution time: 00:00:04.004

(No column name)
7


**13.  It must be their brains, not their looks: Find the name and grade of all students who are liked by more than one other student.**

In [18]:
WITH Liked(liked, ID2)
AS(
    SELECT count(ID1) as liked, ID2 int FROM Likes GROUP BY ID2
)

SELECT name, grade FROM Highschooler WHERE ID in (SELECT ID2 FROM Liked L WHERE L.liked >= 2);

(2 rows affected)

Total execution time: 00:00:00.122

name,grade
Cassandra,9
Kris,10


**14.  Most gregarious: Find the name and grade of the student(s) with the greatest number of friends.**

In [19]:

DECLARE @tabla1 as TABLE (id int, numero int)
INSERT INTO @tabla1 SELECT ID1, count(ID2) FROM Friend GROUP BY ID1

SELECT name, grade FROM Highschooler WHERE ID in (SELECT ID FROM @tabla1 WHERE numero = (SELECT max(numero) FROM @tabla1));


(16 rows affected)

(2 rows affected)

Total execution time: 00:00:00.206

name,grade
Andrew,10
Alexis,11


# Data Modification Statements#

Each of the questions in this section (except the first) assumes the database is in the state after the modifications from the previous question. If you find you've made a mistake or want to restart, just reload the schema and data from scratch.

**15.  End of the school year: Increment the year of all students by 1, such that all 9th graders become 10th graders, all 10th graders become 11th graders, and so on. To check your modification, find the names of all students who are in a grade higher than 12 (we'll deal with them later).**

In [20]:
UPDATE Highschooler set grade = grade + 1;

SELECT name, grade from Highschooler where grade>12

(16 rows affected)

(4 rows affected)

Total execution time: 00:00:00.183

name,grade
Jordan,13
John,13
Kyle,13
Logan,13


**16.  Graduating seniors: After the previous update, some students ended up in 13th grade. They're actually graduating, so remove all tuples in Highschooler with a grade higher than 12. To check your modification, find the total number of students remaining in the Highschooler table.**

In [21]:
DELETE FROM Highschooler WHERE grade > 12;

SELECT COUNT(ID) from Highschooler


(4 rows affected)

(1 row affected)

Total execution time: 00:00:00.113

(No column name)
12


**17.  Now clean up this mess: While friendships may continue after graduation, we can't model them for students no longer in our database. Remove all tuples in Friend and all tuples in Likes that refer to nonexistent students. (This will require two separate commands.) To check your modification, find the names of all students whom Austin likes or who are friends with Austin.**

In [22]:
DELETE FROM Friend WHERE ID1 not in (SELECT ID FROM Highschooler) AND ID2 not in (SELECT ID FROM Highschooler);
DELETE FROM Likes WHERE ID1 not in (SELECT ID FROM Highschooler) AND ID2 not in (SELECT ID FROM Highschooler);
SELECT H2.name FROM Highschooler H1, Friend, Highschooler H2 WHERE H1.name = 'Austin' AND H1.ID = Friend.ID1 AND H2.ID = Friend.ID2;


(6 rows affected)

(0 rows affected)

(1 row affected)

Total execution time: 00:00:02.389

name
Andrew


**18.  Summertime: Now that they're on vacation, our students have more time to spend with one another. As a result, their social circles have expanded: For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. (Do not add duplicate friendships, friendships that already exist, or friendships with oneself.) To check your modification, for the now 10th-grader Jordan, return the names of all her friends.**

In [25]:
INSERT INTO Friend
SELECT DISTINCT F1.ID1, F2.ID2 FROM Friend F1, Friend F2
WHERE F1.ID2 = F2.ID1 AND F1.ID1<>F2.ID2 AND F1.ID1 NOT IN (SELECT F3.ID1 FROM Friend F3 WHERE F3.ID2=F2.ID2);

SELECT H2.name FROM Highschooler H1, Friend F, Highschooler H2 WHERE H1.name = 'Jordan' AND H1.ID = F.ID1 AND H2.ID = F.ID2;


(82 rows affected)

(11 rows affected)

Total execution time: 00:00:00.696

name
Gabriel
Tiffany
Cassandra
Haley
Andrew
Kris
Brittany
Alexis
Austin
Gabriel
